In [1]:
import os
print([i for i in os.listdir() if 'parquet' in i])

['dataframe.parquet']


In [2]:
spark=SparkSession.builder.getOrCreate()
df=spark.read.load('dataframe.parquet',format='parquet')
df.toPandas().head()

,0,1,2,3,4,5,6,7,8,9,...,1994,1995,1996,1997,1998,1999,filename,label,fs,k
0,0.716302,0.740627,0.599815,0.664514,0.786073,0.553410,0.719064,0.703307,0.607023,0.754069,...,0.631046,0.338797,0.639984,0.530173,0.377894,0.593133,extrahls__201101070953.wav,extrahls,367.5,2000
1,0.739262,0.617246,0.803137,0.462727,0.489625,0.699909,0.725328,0.673239,0.572426,0.744569,...,0.664340,0.480305,0.640480,0.608836,0.620766,0.611966,extrahls__201101091153.wav,extrahls,367.5,2000
2,0.913559,0.779044,0.818879,0.500293,0.702190,0.744516,0.643676,0.867989,0.750828,0.856806,...,0.602872,0.318743,0.666170,0.741681,0.656872,0.615473,extrahls__201101152255.wav,extrahls,367.5,2000
3,0.695254,0.564887,0.662952,0.761360,0.728754,0.759764,0.736816,0.707393,0.781271,0.731554,...,0.597531,0.661305,0.667166,0.641461,0.588037,0.717878,extrahls__201101160804.wav,extrahls,367.5,2000
4,0.822854,0.702048,0.398874,0.642867,0.578157,0.544518,0.740318,0.649023,0.726241,0.641135,...,0.572833,0.666673,0.691166,0.456296,0.648299,0.541012,extrahls__201101160808.wav,extrahls,367.5,2000


In [3]:
#separacion de train_set y test_set

DUMMY_FRAC=1. #usar un set de datos pequeño para prueba de funcionalidad del modelo (1/4) o completo (1.0)
SEED=1234
DROPLAST=True

train_set,test_set=df.sample(withReplacement=False,fraction=DUMMY_FRAC,seed=SEED).randomSplit([0.75,0.25])

k=int(df.select('k').groupBy().avg().collect()[0]['avg(k)'])
f_cols=[str(i) for i in range(int(k))]
n_train=train_set.select('k').count()
n_test=test_set.select('k').count()

if DROPLAST:
    cats=df.select('label').distinct().count()-1
else:
    cats=df.select('label').distinct().count()
print('train_set: %d datos, test_set: %d datos, categorias: %d'%(n_train,n_test,cats))

train_set: 69 datos, test_set: 15 datos, categorias: 2


In [4]:
import numpy as np
import h5py as h5
from keras.models import Sequential
from keras.layers import Conv1D,MaxPooling1D, Dropout,Dense,Activation,Flatten
from keras.utils import to_categorical

Using TensorFlow backend.


In [79]:
from pyspark.ml.feature import VectorAssembler,StringIndexer,OneHotEncoder,PCA
from pyspark.ml import Pipeline

pca_k=250

#preprocesamiento de datos de entrada
vecAssembler = VectorAssembler(inputCols=f_cols, outputCol="freq_channels_")
vecAssembler0 = VectorAssembler(inputCols=f_cols, outputCol="freq_channels")
pca=PCA(k=pca_k,inputCol='freq_channels_',outputCol='freq_channels')

#preprocesamiento de datos de salida
Indexer = StringIndexer(inputCol='label',outputCol='labelIndex')
OneHot= OneHotEncoder(inputCol='labelIndex',outputCol='out',dropLast=DROPLAST)


In [80]:
import numpy as np
import pandas as pd
from keras.optimizers import Adagrad
from keras.regularizers import l2

if pca_k>0:
    x_PreProc=Pipeline(stages=[vecAssembler,pca])
else:
    x_PreProc=Pipeline(stages=[vecAssembler0])
    pca_k=k

x_PreProc=x_PreProc.fit(train_set)

y_PreProc=Pipeline(stages=[Indexer,OneHot])
y_PreProc=y_PreProc.fit(train_set)

x_train=np.array(x_PreProc.transform(train_set).select('freq_channels').collect())
x_train=x_train.reshape((n_train,pca_k,1))

y_train=np.array(y_PreProc.transform(train_set).select('out').collect(),dtype='int32')
y_train=y_train.reshape((n_train,cats))

x_test=np.array(x_PreProc.transform(test_set).select('freq_channels').collect())
x_test=x_test.reshape((n_test,pca_k,1))

y_test=np.array(y_PreProc.transform(test_set).select('out').collect(),dtype='int32')
y_test=y_test.reshape(n_test,cats)
print('pca_k : %d'%pca_k)

pca_k : 250


In [81]:
'''
----------------------------------------------------------------------
filters 102, batch_size 5, kernel_size 178, n_hidden 50, pool_size: 51
accuracy 0.7841, recall 0.8333 , 0.8750
----------------------------------------------------------------------
'''

from sklearn.metrics import recall_score
vec_round=np.vectorize(lambda x: int(round(x)))

RANDOM_SEARCH=0
filters=219
kernel_size=130
pool_size=24
batch_size=35
rate=0.4
activation='softmax'
activation_l='softmax'
n_hidden=143
l2_r=0.
l2_cnn=0.
l2_r2=0.0001

accuracy=0.0
best_accuracy=0
best_params=None

epochs= 40 if RANDOM_SEARCH else 400
acc_target = 0.88 if RANDOM_SEARCH else 0.01
i=0

while (accuracy < acc_target)|(i<100): 
    
    if RANDOM_SEARCH:
        filters=int(np.random.uniform(80,250))
        kernel_size=int(np.random.uniform(2,250))
        n_hidden=int(np.random.uniform(5,150))
        pool_size=int(np.random.uniform(2,40))
        batch_size=int(np.random.uniform(1,6))
    
    model=Sequential()
    
    model.add(Conv1D(filters=filters,kernel_size=kernel_size,padding='causal',activation=activation_l,kernel_regularizer=l2(l2_cnn),input_shape=(pca_k,1)))
    model.add(MaxPooling1D(pool_size=pool_size))
    model.add(Dropout(rate=rate))
    
    model.add(Flatten())
    
    model.add(Dense(n_hidden,kernel_regularizer=l2(l2_r)))
    model.add(Activation(activation))
    
    model.add(Dense(cats,kernel_regularizer=l2(l2_r2)))
    model.add(Activation(activation))

    
    model.compile(loss='categorical_crossentropy',optimizer=Adagrad(),metrics=['accuracy'])
    model.fit(x_train,y_train,batch_size=batch_size,epochs=epochs,verbose=(not RANDOM_SEARCH),validation_data=(x_test,y_test))
            
    evaluation=model.evaluate(x_test,y_test)
    evaltrain=model.evaluate(x_train,y_train)
    yt_pred=vec_round(model.predict(x_test))
    recall1,recall2=recall_score(y_test[:,0],yt_pred[:,0]),recall_score(y_test[:,1],yt_pred[:,1])
    
    accuracy=(evaluation[1]+evaltrain[1])/2
    if accuracy>=best_accuracy:
        best_accuracy=accuracy
        best_params={'filters': filters,'batch_size': batch_size,'kernel_size':kernel_size,'n_hidden':n_hidden,'pool_size':pool_size}
        print('\n----------------------------------------------------------------------')
        print('filters %d, batch_size %d, kernel_size %d, n_hidden %d, pool_size: %d'%(best_params['filters'],best_params['batch_size'],best_params['kernel_size'],best_params['n_hidden'],best_params['pool_size']))
        print('accuracy %1.4f, recall %1.4f , %1.4f'%(best_accuracy,recall1,recall2))
        print('----------------------------------------------------------------------\n')
    print('TRAIN: loss= %1.5f, accuracy= %1.5f |TEST: loss= %1.5f,accuracy= %1.5f)\n'%(evaltrain[0],evaltrain[1],evaluation[0],evaluation[1]))
    if not RANDOM_SEARCH:
        i=100
    else:
        i+=1
    
print('******************************')
print(best_params)
print(best_accuracy)

Train on 69 samples, validate on 15 samples
Epoch 1/400
69/69 [==============================] - 9s - loss: 0.5149 - acc: 0.3333 - val_loss: 0.6490 - val_acc: 0.4667
Epoch 2/400
69/69 [==============================] - 0s - loss: 0.5120 - acc: 0.6667 - val_loss: 0.6535 - val_acc: 0.4667
Epoch 3/400
69/69 [==============================] - 0s - loss: 0.5128 - acc: 0.6667 - val_loss: 0.6561 - val_acc: 0.4667
Epoch 4/400
69/69 [==============================] - 0s - loss: 0.5100 - acc: 0.6667 - val_loss: 0.6577 - val_acc: 0.4667
Epoch 5/400
69/69 [==============================] - 0s - loss: 0.5103 - acc: 0.6667 - val_loss: 0.6575 - val_acc: 0.4667
Epoch 6/400
69/69 [==============================] - 0s - loss: 0.5089 - acc: 0.6667 - val_loss: 0.6562 - val_acc: 0.4667
Epoch 7/400
69/69 [==============================] - 0s - loss: 0.5123 - acc: 0.6812 - val_loss: 0.6562 - val_acc: 0.4667
Epoch 8/400
69/69 [==============================] - 0s - loss: 0.5063 - acc: 0.6667 - val_loss: 0.655

69/69 [==============================] - 0s - loss: 0.2600 - acc: 0.7681 - val_loss: 0.5316 - val_acc: 0.6667
Epoch 68/400
69/69 [==============================] - 0s - loss: 0.2611 - acc: 0.7826 - val_loss: 0.5332 - val_acc: 0.6667
Epoch 69/400
69/69 [==============================] - 0s - loss: 0.2546 - acc: 0.7826 - val_loss: 0.5127 - val_acc: 0.8000
Epoch 70/400
69/69 [==============================] - 0s - loss: 0.2542 - acc: 0.8116 - val_loss: 0.5160 - val_acc: 0.8000
Epoch 71/400
69/69 [==============================] - 0s - loss: 0.2608 - acc: 0.8261 - val_loss: 0.5133 - val_acc: 0.8000
Epoch 72/400
69/69 [==============================] - 0s - loss: 0.2592 - acc: 0.7971 - val_loss: 0.5088 - val_acc: 0.8000
Epoch 73/400
69/69 [==============================] - 0s - loss: 0.2522 - acc: 0.7536 - val_loss: 0.5261 - val_acc: 0.6667
Epoch 74/400
69/69 [==============================] - 0s - loss: 0.2575 - acc: 0.7971 - val_loss: 0.5830 - val_acc: 0.6000
Epoch 75/400
69/69 [=========

69/69 [==============================] - 0s - loss: 0.2089 - acc: 0.7681 - val_loss: 0.4919 - val_acc: 0.8000
Epoch 134/400
69/69 [==============================] - 0s - loss: 0.2090 - acc: 0.7826 - val_loss: 0.5004 - val_acc: 0.8000
Epoch 135/400
69/69 [==============================] - 0s - loss: 0.2090 - acc: 0.8116 - val_loss: 0.4861 - val_acc: 0.7333
Epoch 136/400
69/69 [==============================] - 0s - loss: 0.2069 - acc: 0.8116 - val_loss: 0.4916 - val_acc: 0.8000
Epoch 137/400
69/69 [==============================] - 0s - loss: 0.2085 - acc: 0.7826 - val_loss: 0.4883 - val_acc: 0.8000
Epoch 138/400
69/69 [==============================] - 0s - loss: 0.2065 - acc: 0.7971 - val_loss: 0.4923 - val_acc: 0.8000
Epoch 139/400
69/69 [==============================] - 0s - loss: 0.2059 - acc: 0.8116 - val_loss: 0.4919 - val_acc: 0.8000
Epoch 140/400
69/69 [==============================] - 0s - loss: 0.2075 - acc: 0.7826 - val_loss: 0.4865 - val_acc: 0.8000
Epoch 141/400
69/69 [=

69/69 [==============================] - 0s - loss: 0.1822 - acc: 0.7826 - val_loss: 0.4763 - val_acc: 0.8000
Epoch 200/400
69/69 [==============================] - 0s - loss: 0.1826 - acc: 0.8261 - val_loss: 0.4770 - val_acc: 0.8000
Epoch 201/400
69/69 [==============================] - 0s - loss: 0.1831 - acc: 0.8261 - val_loss: 0.4794 - val_acc: 0.8000
Epoch 202/400
69/69 [==============================] - 0s - loss: 0.1838 - acc: 0.7826 - val_loss: 0.4815 - val_acc: 0.8000
Epoch 203/400
69/69 [==============================] - 0s - loss: 0.1824 - acc: 0.7971 - val_loss: 0.4770 - val_acc: 0.8000
Epoch 204/400
69/69 [==============================] - 0s - loss: 0.1809 - acc: 0.8261 - val_loss: 0.4780 - val_acc: 0.8000
Epoch 205/400
69/69 [==============================] - 0s - loss: 0.1822 - acc: 0.8406 - val_loss: 0.4913 - val_acc: 0.8000
Epoch 206/400
69/69 [==============================] - 0s - loss: 0.1809 - acc: 0.8261 - val_loss: 0.4843 - val_acc: 0.8000
Epoch 207/400
69/69 [=

69/69 [==============================] - 0s - loss: 0.1653 - acc: 0.8406 - val_loss: 0.4742 - val_acc: 0.8000
Epoch 266/400
69/69 [==============================] - 0s - loss: 0.1651 - acc: 0.7826 - val_loss: 0.4726 - val_acc: 0.8000
Epoch 267/400
69/69 [==============================] - 0s - loss: 0.1651 - acc: 0.8116 - val_loss: 0.4769 - val_acc: 0.8000
Epoch 268/400
69/69 [==============================] - 0s - loss: 0.1654 - acc: 0.8116 - val_loss: 0.4748 - val_acc: 0.8000
Epoch 269/400
69/69 [==============================] - 0s - loss: 0.1643 - acc: 0.7826 - val_loss: 0.4722 - val_acc: 0.8000
Epoch 270/400
69/69 [==============================] - 0s - loss: 0.1647 - acc: 0.8116 - val_loss: 0.4789 - val_acc: 0.8000
Epoch 271/400
69/69 [==============================] - 0s - loss: 0.1641 - acc: 0.7971 - val_loss: 0.4737 - val_acc: 0.8000
Epoch 272/400
69/69 [==============================] - 0s - loss: 0.1643 - acc: 0.7971 - val_loss: 0.4706 - val_acc: 0.8000
Epoch 273/400
69/69 [=

69/69 [==============================] - 0s - loss: 0.1520 - acc: 0.7826 - val_loss: 0.4659 - val_acc: 0.8000
Epoch 332/400
69/69 [==============================] - 0s - loss: 0.1519 - acc: 0.8696 - val_loss: 0.4635 - val_acc: 0.8000
Epoch 333/400
69/69 [==============================] - 0s - loss: 0.1516 - acc: 0.7826 - val_loss: 0.4659 - val_acc: 0.8000
Epoch 334/400
69/69 [==============================] - 0s - loss: 0.1515 - acc: 0.7971 - val_loss: 0.4649 - val_acc: 0.8000
Epoch 335/400
69/69 [==============================] - 0s - loss: 0.1519 - acc: 0.8261 - val_loss: 0.4643 - val_acc: 0.8000
Epoch 336/400
69/69 [==============================] - 0s - loss: 0.1516 - acc: 0.8116 - val_loss: 0.4690 - val_acc: 0.8000
Epoch 337/400
69/69 [==============================] - 0s - loss: 0.1509 - acc: 0.8116 - val_loss: 0.4654 - val_acc: 0.8000
Epoch 338/400
69/69 [==============================] - 0s - loss: 0.1509 - acc: 0.8261 - val_loss: 0.4663 - val_acc: 0.8000
Epoch 339/400
69/69 [=

69/69 [==============================] - 0s - loss: 0.1415 - acc: 0.7826 - val_loss: 0.4711 - val_acc: 0.8000
Epoch 398/400
69/69 [==============================] - 0s - loss: 0.1407 - acc: 0.8406 - val_loss: 0.4713 - val_acc: 0.8000
Epoch 399/400
69/69 [==============================] - 0s - loss: 0.1408 - acc: 0.7681 - val_loss: 0.4695 - val_acc: 0.8000
Epoch 400/400
32/69 [============>.................] - ETA: 0s
----------------------------------------------------------------------
filters 219, batch_size 35, kernel_size 130, n_hidden 143, pool_size: 24
accuracy 0.7986, recall 0.8333 , 0.8750
----------------------------------------------------------------------

TRAIN: loss= 0.14021, accuracy= 0.79710 |TEST: loss= 0.46781,accuracy= 0.80000)

******************************
{'batch_size': 35, 'filters': 219, 'pool_size': 24, 'n_hidden': 143, 'kernel_size': 130}
0.798550730598


In [153]:
X=x_train[:,:,0]
Y0=y_train[:,0]
Y1=y_train[:,1]

from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf0 = clf.fit(X, Y0)
clf1 = clf.fit(X, Y1)
treeRes=(clf.score(x_test[:,:,0],y_test[:,0]),clf.score(x_test[:,:,0],y_test[:,1]))
print('tree classifier score: %1.3f'%((treeRes[0]+treeRes[1])/2))

from sklearn.ensemble import GradientBoostingClassifier
clf0 = GradientBoostingClassifier(n_estimators=1000, learning_rate=1.0,max_depth=1, random_state=0).fit(X, Y0)
clf1 = GradientBoostingClassifier(n_estimators=1000, learning_rate=1.0,max_depth=1, random_state=0).fit(X, Y1)
GTBresult=(clf0.score(x_test[:,:,0],y_test[:,0]),clf1.score(x_test[:,:,0],y_test[:,1]))
print('Gradient Boosting Classifier score: %1.3f'%((GTBresult[0]+GTBresult[1])/2))

print('Conv Neural network score: %1.3f'%evaluation[1])

tree classifier score: 0.533
Gradient Boosting Classifier score: 0.600
Conv Neural network score: 0.800


In [85]:
from bokeh.plotting import figure,show
from bokeh.io import output_notebook
from bokeh.layouts import row
output_notebook()
xp=x_test
yp=y_test

y_pred=model.predict(xp)

p=figure(title='Output 1: normal/extrahls',x_axis_label='true result',y_axis_label='model result')
p.scatter(x=yp[:,0],y=y_pred[:,0],color='blue')
q=figure(title='Output 2: murmur/extrahls',x_axis_label='true result',y_axis_label='model result')
q.scatter(x=yp[:,1],y=y_pred[:,1],color='red')
if DROPLAST:
    show(row([p,q]))
else:
    r=figure()
    r.scatter(x=yp[:,2],y=y_pred[:,2],color='green')
    show(row([p,q,r]))


Loading BokehJS ...

In [88]:
from sklearn.metrics import confusion_matrix

confusion=[]
th=[0.5,0.5]
def vec_round(x,th):
    v=np.vectorize(lambda x: 1 if x>=th else 0)
    return v(x)

for cat in [0,1]:    
    y_predBin=vec_round(y_pred[:,cat].flatten(),th[cat])
    cm=confusion_matrix(yp[:,cat].flatten(),y_predBin)
    cm=cm/sum(cm.flatten())
    confusion.append(cm)
    print('confussion matrix for output %d'%(cat+1))
    print(cm)
    print('recall: %1.4f'%(1-cm[0,1]/(cm[0,1]+cm[1,1])))
    print('precision: %1.4f\n'%(1-cm[1,0]/(cm[1,0]+cm[1,1])))

confussion matrix for output 1
[[ 0.53333333  0.06666667]
 [ 0.06666667  0.33333333]]
recall: 0.8333
precision: 0.8333

confussion matrix for output 2
[[ 0.33333333  0.13333333]
 [ 0.06666667  0.46666667]]
recall: 0.7778
precision: 0.8750



In [89]:
from sklearn.metrics import precision_recall_curve
p=figure(title='ROC Curve',x_range=(0,1),y_range=(0,1),y_axis_label='recall',x_axis_label='1-precision')

precision0, recall0, thresholds0 = precision_recall_curve(yp[:,0],y_pred[:,0])
precision1, recall1, thresholds1 = precision_recall_curve(yp[:,1],y_pred[:,1])

p.line(y=recall0,x=1-precision0,color='blue',legend='normal/extrahls')
p.line(y=recall1,x=1-precision1,color='red',legend='murmur/extrahls')
p.line(x=[0,1],y=[0,1],color='black')

show(p)
